In [1]:
#preprocesamiento datos de aprehendidos/detenidos
import pandas as pd
import numpy as np

#CARGA DE DATOS

nombre_archivo = 'detenidosaprehendidos/dataset/mdi_detenidosaprehendidos_pm_2025_enero_octubre.xlsx'

try:
    print("Cargando archivo Excel... esto puede tardar unos segundos...")
    df_apre = pd.read_excel(
        nombre_archivo,
        sheet_name=1,
        dtype={'codigo_parroquia': str},
        engine='openpyxl'
    )
    print(f"Registros totales: {len(df_apre)}")

except Exception as e:
    print("Error cargando el Excel:", e)



Cargando archivo Excel... esto puede tardar unos segundos...
Error cargando el Excel: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.


In [ ]:
#LIMPIEZA DE COORDENADAS

for col in ["latitud", "longitud"]:
    df_apre[col] = (
        df_apre[col]
        .astype(str)
        .str.replace(",", ".", regex=False)
    )
    df_apre[col] = pd.to_numeric(df_apre[col], errors="coerce")

# Eliminar coordenadas inválidas (nulas y el punto de origen)
df_apre = df_apre[
    df_apre[["latitud", "longitud"]].notna().all(axis=1)  
    & (df_apre["latitud"] != 0)
    & (df_apre["longitud"] != 0)
].copy()

In [ ]:
#PROCESAR FECHA Y HORA

try:
    df_apre["fecha"] = pd.to_datetime(df_apre["fecha_detencion_aprehension"], errors="coerce")

    #limpiar la columna de hora
    df_apre["hora_limpia"] = (
        df_apre["hora_detencion_aprehension"]
        .astype(str)
        .str.replace(" ", "")
        .str.strip()
    )

    #combinar fecha y hora
    df_apre["fecha_completa"] = pd.to_datetime(
        df_apre["fecha"].astype(str) + " " + df_apre["hora_limpia"],
        errors="coerce"
    )

except Exception as e:
    print("Advertencia procesando fecha/hora:", e)